In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import time

# Load and prepare data
patient_df = pd.read_csv("primepatient_dataset.csv")
X = patient_df.drop('outcome', axis=1).values
y = patient_df['outcome'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [2]:
# Function to create and train models
def train_model(X_train, y_train, X_test, y_test, data_size, hidden_layers, nodes_per_layer):
    # Limit data size
    X_train_subset = X_train[:data_size]
    y_train_subset = y_train[:data_size]

    # Create model
    model = Sequential()

    # Input layer
    model.add(Dense(nodes_per_layer, activation='relu', input_shape=(X_train.shape[1],)))

    # Hidden layers
    for _ in range(hidden_layers - 1):
        model.add(Dense(nodes_per_layer, activation='relu'))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    # Train model and measure time
    start_time = time.time()

    history = model.fit(X_train_subset, y_train_subset,
                       epochs=100,
                       batch_size=32,
                       verbose=0,
                       validation_split=0.2)

    end_time = time.time()
    execution_time = end_time - start_time

    # Get final training and validation errors
    train_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]

    return train_loss, val_loss, execution_time

In [3]:
# Run experiments
results = []
configurations = [
    (1000, 1, 4),
    (10000, 1, 4),
    (100000, 1, 4),
    (1000, 2, 4),
    (10000, 2, 4),
    (100000, 2, 4)
]

for data_size, hidden_layers, nodes in configurations:
    # Handle case where requested data size is larger than available data
    actual_size = min(data_size, len(X_train_scaled))

    train_loss, val_loss, exec_time = train_model(
        X_train_scaled, y_train, X_test_scaled, y_test,
        actual_size, hidden_layers, nodes
    )

    results.append({
        'Data size': actual_size,
        'Configuration': f'{hidden_layers} hidden layer{"s" if hidden_layers > 1 else ""} {nodes} nodes',
        'Training error': train_loss,
        'Validation error': val_loss,
        'Time of execution': exec_time
    })

# Display results
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 Data size           Configuration  Training error  Validation error  Time of execution
      1000  1 hidden layer 4 nodes        0.061406          0.043239          17.512232
     10000  1 hidden layer 4 nodes        0.008779          0.011313          54.558929
    100000  1 hidden layer 4 nodes        0.003282          0.003973         427.563161
      1000 2 hidden layers 4 nodes        0.027271          0.028756          14.162219
     10000 2 hidden layers 4 nodes        0.004131          0.008034          56.262301
    100000 2 hidden layers 4 nodes        0.003221          0.003501         454.291734
